Кобзарь О.С. Хабибуллин Р.А. 31.07.2019

# Работа с исходными данными

* Выгрузка исходных данных со СУ и построение графиков
* Обработка исходных данных с разнесением параметров по столбцам и сохранение изменений в отдельный файл
* Загрузка обработанных данных и построение по ним графиков для проверки 

Для работы необходимы библиотеки: `pandas`, `plotly`, `datetime`, `numpy`

Также необходимо изменять названия файлов для импорта и экспорта данных на требуемые

In [ ]:
import pandas as pd
import numpy as np

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
init_notebook_mode(connected=True)

from datetime import date

Загрузка csv без заголовков

In [ ]:
csv_file_name = "kobzar1.csv"

In [ ]:
cs_data = pd.read_csv(csv_file_name,sep=';', header=None)

In [ ]:
cs_data.head()

In [ ]:
cs_data.shape

Лист с названиями параметрами

In [ ]:
list_parametrs = np.unique(cs_data[1])

Ключ для параметра

In [ ]:
list_parametrs[0]

Число параметров

In [ ]:
len(list_parametrs)

Номер для каждого параметра и количество записей

In [ ]:
def get_information(cs_data, list_parametrs):
    for i in range(len(list_parametrs)):
        amount_of_parametr_values = len(cs_data[cs_data[1] == list_parametrs[i]])
        print("№" + str (i)+ " Кол-во записей: " + str(amount_of_parametr_values) +'  ' +str(list_parametrs[i]))

In [ ]:
get_information(cs_data, list_parametrs)

Построение графика для одного параметра по его номеру, используя весь DataFrame

In [ ]:
# Create a trace
number = 19
specific_param_by_number = (cs_data[cs_data[1] == list_parametrs[number]])
print(list_parametrs[number])
trace = go.Scattergl(
    x = specific_param_by_number[2].values,
    y = specific_param_by_number[3].values,
    mode = 'markers'
)

data = [trace]

# Plot and embed in ipython notebook!
iplot(data, filename='basic-scatter')

Функции для редактирования исходных данных со станции управления

In [ ]:
def initial_editing(data):
    del data[0]
    str_to_delete = "КУСТ 18. ГЗУ 18А. Скважина 1354."
    data[1] = data[1].str.replace(str_to_delete,"")
    data.index = data[2]
    del data[2]
    data.index = pd.to_datetime(data.index)
    return data

def extract_df_one_parametr_and_edit(data, list_of_params, number_of_param_in_list):
    extracted_df_one_param = data[data[1] == list_of_params[number_of_param_in_list]].copy()
    edited_df_one_param = extracted_df_one_param.rename(index=str, columns = {3: extracted_df_one_param[1][1]})
    del edited_df_one_param[1]
    return edited_df_one_param

def create_edited_df(data, parametrs_list):
    init_one_parametr_df = extract_df_one_parametr_and_edit(data, parametrs_list, 0)
    result = init_one_parametr_df
    for i in range(1, len(parametrs_list)):
        new_one_parametr_df = extract_df_one_parametr_and_edit(data, parametrs_list, i)
        result = result.join([new_one_parametr_df], how = "outer", sort=True)
    return result

Пример использования данных функций на маленькой вырезке исходных данных

Сперва посмотрим на исходные данные

In [ ]:
cs_data_try_merge = cs_data[-1000:].copy()
cs_data_try_merge.head()

Выполним изменения, посмотрим на готовый DataFrame и запишем его в .csv

Если возникает ошибка, значит Вы хотите совершить преобразование повторно на уже обработанном DataFrame

In [ ]:

cs_data_try_merge = initial_editing(cs_data_try_merge)

small_list_parametrs = np.unique(cs_data_try_merge[1])
get_information(cs_data_try_merge, small_list_parametrs)
result_by_func = create_edited_df(cs_data_try_merge, small_list_parametrs)

result_by_func.to_excel("check2.xlsx")
result_by_func.head()

Проведем подобную операцию уже с большим исходным DataFrame и запишем данные

*Внимание! Будет расчет и запись будет производится долго*

In [ ]:
cs_big_data_try_merge = cs_data.copy()
cs_big_data_try_merge = initial_editing(cs_big_data_try_merge)
big_list_parametrs = np.unique(cs_big_data_try_merge[1])
big_result_by_func = create_edited_df(cs_big_data_try_merge, big_list_parametrs)
big_result_by_func.to_csv("edited_kobzar2.csv")


Ради проверки прочтем записанные данные, посмотрим на них и построим графики

In [ ]:
test = pd.read_csv("edited_kobzar2.csv")
test.index = pd.to_datetime(test["2"])
test.head()

In [ ]:
test.info()

Из обработанных данных выберем промежуток времени и построим по нему график

*Т.к. данных много, построение графика не быстрое*

In [ ]:
year_start = 2019
month_start = 2
day_start = 21
hour_start = 0
date_from = pd.to_datetime(pd.DataFrame({'year': [year_start],
                           'month': [month_start],
                           'day': [day_start],
                            'hour': [hour_start]}))
date_from.iloc[0]

In [ ]:
test_month = test[test.index > date_from.iloc[0]]

In [ ]:
for number, name in enumerate(test_month.columns):
    print(str(number) + " " + name)

По оси горизонтальной время - индекс DataFrame, а по оси y значения из столбцов, к которым можно обратиться по номерам, перечисленным выше

In [ ]:
trace1_column_name = test_month.columns[34]
trace1 = go.Scattergl(
    x = test_month.index,
    y = test_month[trace1_column_name],
    name = trace1_column_name,
    mode = 'markers'
)

trace2_column_name = test_month.columns[37]
trace2 = go.Scattergl(
    x = test_month.index,
    y = test_month[trace2_column_name],
    name = trace2_column_name,
    mode = 'markers'
)
trace3_column_name = test_month.columns[36]
trace3 = go.Scattergl(
    x = test_month.index,
    y = test_month[trace3_column_name],
    name = trace1_column_name,
    mode = 'markers'
)


data = [trace1, trace2, trace3]


layout = dict(title = "df"
             )

fig = dict(data=data, layout=layout)

iplot(fig, filename='basic-scatter')